In [3]:
import requests 
import json
import os
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.append('../src')
import apilast as als
import sqltools as sqt
import sqlalchemy as alch
import cleansing as cls
from pymongo import MongoClient
import pymongo

In [4]:
keylast = os.getenv("keylast")

In [5]:
client = MongoClient("localhost:27017")

In [6]:
client.list_database_names()

['First_try', 'admin', 'config', 'ironhack', 'lastusers', 'local', 'musicable']

In [7]:
lastusers = client.get_database("lastusers")

In [8]:
lastusers.list_collection_names()

['AveKaesar', 'usuarios', 'sinatxester']

In [9]:
lastusers.usuarios.insert_one({'hola':'prueba','flipa':'prueba'})

In [10]:
lastuser = 'AveKaesar'

In [12]:
new_user = lastusers.create_collection(lastuser)

In [45]:
#totscrobles

In [72]:
url = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={lastuser}&limit=1000&api_key={keylast}&format=json'
req = requests.get(url).json()

In [85]:
lastuser = 'sinatxester'

In [82]:
new_user = lastusers.create_collection(lastuser)

In [83]:
lastuser = 'jesteruki'

In [86]:
url = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={lastuser}&limit=1000&api_key={keylast}&format=json'
req = requests.get(url).json()['recenttracks']
pages = int(req['@attr']['totalPages'])
print(f'recovering {pages} pages')


for i in range(1,pages+1):
    page = i
    url_t = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={lastuser}&limit=1000&page={page}&api_key={keylast}&format=json'
    req_t = requests.get(url_t).json()['recenttracks']['track']
    for r in req_t:
        #print(r)
        lastusers.sinatxester.insert_one(r)
    
    if i%5==0:
        print(f'pag {i} done')

recovering 76 pages
pag 5 done
pag 10 done
pag 15 done
pag 20 done
pag 25 done
pag 30 done
pag 35 done
pag 40 done
pag 45 done
pag 50 done
pag 55 done
pag 60 done
pag 65 done
pag 70 done
pag 75 done


In [87]:
lastusers.list_collection_names()

['AveKaesar', 'usuarios', 'sinatxester']

In [13]:
ester = lastusers.get_collection("sinatxester")

In [14]:
ester.find_one()

{'_id': ObjectId('6217b85175bd69ba0374287e'),
 'artist': {'mbid': 'e795e03d-b5d5-4a5f-834d-162cfb308a2c',
  '#text': 'PJ Harvey'},
 'streamable': '0',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/6bc1781107a6ff90158c0ab222188b0d.jpg'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/6bc1781107a6ff90158c0ab222188b0d.jpg'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/6bc1781107a6ff90158c0ab222188b0d.jpg'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/6bc1781107a6ff90158c0ab222188b0d.jpg'}],
 'mbid': '0a684eac-dabc-3b19-a3aa-9dbb8cd9cf43',
 'album': {'mbid': '1e9f36a7-074e-4a84-a6b2-c3ff82d7d1e1',
  '#text': 'Stories From the City, Stories From the Sea'},
 'name': 'Good Fortune',
 'url': 'https://www.last.fm/music/PJ+Harvey/_/Good+Fortune',
 'date': {'uts': '1420113107', '#text': '01 Jan 2015, 11:51'}}

In [15]:
len(ester.distinct('date.uts'))

170179

In [16]:

numerito = 0
for ut in lastusers.sinatxester.distinct('date.uts'):
    if int(ut)>numerito:
        numerito = int(ut)

In [17]:
numerito

1645713076

In [22]:
#creamos funcioncita
def utsmongo(colecion):
    '''
    recibe un colección de una base de datos en mongo (usuario específico en lastusers)
    devuelve el mayor uts de la colección.
    '''
    numerito = 0
    for ut in colecion.distinct('date.uts'):
        if int(ut)>numerito:
            numerito = int(ut)
    return numerito

In [23]:
utsmongo(ester)

1645713076

In [20]:
cls.utslocal(numerito)

'2022-02-24 15:31:16'

In [21]:
ester.find_one({"date.uts": f"{numerito}"})

{'_id': ObjectId('6217b9ab75bd69ba03759a82'),
 'artist': {'mbid': 'd54ea4a6-0b9c-4e47-bed0-289ae9ff4037',
  '#text': 'Los Chichos'},
 'streamable': '0',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png'}],
 'mbid': '4ccb539e-d661-4b6f-a184-29635571649f',
 'album': {'mbid': '41240f03-c082-4dda-a5b4-8475fd18c706',
  '#text': 'Son Ilusiones'},
 'name': 'Eres falsa y embustera',
 'url': 'https://www.last.fm/music/Los+Chichos/_/Eres+falsa+y+embustera',
 'date': {'uts': '1645713076', '#text': '24 Feb 2022, 14:31'}}

In [28]:

avenum = utsmongo(lastusers.AveKaesar)
avenum

1645707198

In [29]:
cls.utslocal(avenum)

'2022-02-24 13:53:18'

In [30]:
lastusers.AveKaesar.find_one({"date.uts": f"{avenum}"})

{'_id': ObjectId('6217b70475bd69ba0372d467'),
 'artist': {'mbid': 'c5c2ea1c-4bde-4f4d-bd0b-47b200bf99d6', '#text': 'The xx'},
 'streamable': '0',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png'}],
 'mbid': '11be1671-4cb4-418c-a066-426bde94cac7',
 'album': {'mbid': '0732d223-484b-440f-a031-2ff5dd5b2ede',
  '#text': 'I See You'},
 'name': 'A Violent Noise',
 'url': 'https://www.last.fm/music/The+xx/_/A+Violent+Noise',
 'date': {'uts': '1645707198', '#text': '24 Feb 2022, 12:53'}}